In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/PBL7/Tram

Mounted at /content/drive
/content/drive/MyDrive/PBL7/Tram


In [ ]:
!pip install cloudscraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.2 MB/s eta 0:00:00


## 3. Crawl User Profiles

In [ ]:
user_ratings = pd.read_csv('/content/drive/MyDrive/PBL7/Tram/user_ratings.csv', index_col=False)
print(user_ratings.shape)
user_ratings

(182171, 4)


,isbn,user_url,rating,rating_time
0,0889610487,/mybookshelf/_Figment_/,8,2022-04-17T00:00:00
1,0394280741,/mybookshelf/Ella-the-Elf/,7,2003-01-11T00:00:00
2,0394280741,/mybookshelf/AnonymousFinder/,7,2011-04-08T00:00:00
3,0394280741,/mybookshelf/_Figment_/,8,2022-04-17T00:00:00
4,0394280741,/mybookshelf/redhead344/,8,2002-11-17T00:00:00
...,...,...,...,...
182166,0099507659,/mybookshelf/mcsar/,8,2022-05-16T00:00:00
182167,1407159283,/mybookshelf/6of8/,6,2022-01-16T00:00:00
182168,1407159283,/mybookshelf/6of8/,6,2023-10-14T00:00:00
182169,0755309278,/mybookshelf/boschman/,9,2009-03-29T00:00:00


In [ ]:
user_list = user_ratings[['user_url']].drop_duplicates()
print(user_list.shape)
user_list

(41707, 1)


,user_url
0,/mybookshelf/_Figment_/
1,/mybookshelf/Ella-the-Elf/
2,/mybookshelf/AnonymousFinder/
4,/mybookshelf/redhead344/
6,/mybookshelf/cestmoi/
...,...
182149,/mybookshelf/vrenda/
182158,/mybookshelf/cosninocanines/
182160,/mybookshelf/AMORC3339/
182164,/mybookshelf/CelloRocksMyWor/


In [ ]:
count_user_profiles = 30360
user_list = user_list[count_user_profiles:]
print(user_list.shape)
user_list

(11347, 1)


,user_url
119931,/mybookshelf/lmpinto/
119935,/mybookshelf/tigerlily45/
119936,/mybookshelf/grandma8/
119941,/mybookshelf/Dawes/
119942,/mybookshelf/miella/
...,...
182149,/mybookshelf/vrenda/
182158,/mybookshelf/cosninocanines/
182160,/mybookshelf/AMORC3339/
182164,/mybookshelf/CelloRocksMyWor/


In [ ]:
# create user_ratings.csv
# import pandas as pd
# pd.DataFrame(columns=['user_url', 'image_url', 'name', 'area', 'age', 'join_date']).to_csv('user_profiles.csv', mode='w', header=True, index=False)
# # create error_user_urls_profiles.csv
# pd.DataFrame(columns=['user_url']).to_csv('error_user_urls_profiles.csv', mode='w', header=True, index=False)

In [ ]:
import time
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re

error_user_urls = []
user_profiles = []
base_url = 'https://bookcrossing.com'
max_attempts = 10
scraper = cloudscraper.create_scraper()

def save_links_to_csv(user_profiles):
    user_profiles_df = pd.DataFrame(user_profiles, columns=['user_url', 'image_url', 'name', 'area', 'age', 'join_date'])
    user_profiles_df.to_csv('user_profiles.csv', mode='a', header=False, index=False)
def save_error_links_to_csv(error_user_urls):
    error_user_urls_df = pd.DataFrame(error_user_urls, columns=['user_url'])
    error_user_urls_df.to_csv('error_user_urls_profiles.csv', mode='a', header=False, index=False)

for user_url in user_list['user_url']:
    count_user_profiles += 1
    url = base_url + user_url
    for attempt in range(max_attempts):
        try:
            # Lấy nội dung trang web
            response = scraper.get(url)
            # Kiểm tra nếu yêu cầu thành công
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')

                # Trích xuất thông tin với điều kiện kiểm tra
                image_tag = soup.find('div', class_='profile-image').find('img')
                image_url = image_tag['src'] if image_tag else ''
                if image_url.startswith('/'):
                    image_url = base_url + image_url

                detail_data_div = soup.find('div', class_='col small')
                name_tag = detail_data_div.find('h2')
                name = name_tag.text.strip() if name_tag else ''

                area_div = detail_data_div.find('div')
                area = area_div.text.strip().replace('From ', '') if area_div else ''

                age_div = detail_data_div.find('div', text=lambda text: text and 'Age' in text)
                age = age_div.text.strip().replace('Age ', '') if age_div else ''

                join_date_div = detail_data_div.find('div', text=lambda text: text and 'Joined' in text)
                join_date = join_date_div.text.strip().replace('Joined ', '') if join_date_div else ''

                user_profile = {
                    'user_url': user_url,
                    'image_url': image_url,
                    'name': name,
                    'area': area,
                    'age': age,
                    'join_date': join_date
                }
                user_profiles.append(user_profile)

                break
            else:
                print(f"Không thể truy cập trang web, mã trạng thái: {response.status_code}")
                if attempt == max_attempts - 1:
                    print(f"URL: {url} se duoc bo qua.")
                    error_user_urls.append(url)
                    save_error_links_to_csv(error_user_urls)
                    error_user_urls = []
                    break

            # Chờ 3 giây trước khi thử lại
            time.sleep(3)
        except Exception as e:
            print(f"Exception occurred: {e}")
            error_user_urls.append(url)
            save_error_links_to_csv(error_user_urls)
            error_user_urls = []
        finally:
            scraper.close()
    if count_user_profiles % 100 == 0:
        save_links_to_csv(user_profiles)
        print(f"Saved {len(user_profiles)} user profiles. Current Index: {count_user_profiles}.")
        user_profiles = []

# Save any remaining user rprofiles
if user_profiles:
    save_links_to_csv(user_profiles)
    print(f"Saved {len(user_profiles)} remaining user profiles. Total saved: {count_user_profiles}.")

<ipython-input-7-aeaf7edaf765>:45: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  age_div = detail_data_div.find('div', text=lambda text: text and 'Age' in text)
<ipython-input-7-aeaf7edaf765>:48: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  join_date_div = detail_data_div.find('div', text=lambda text: text and 'Joined' in text)


Saved 40 user profiles. Current Index: 30400.
Saved 100 user profiles. Current Index: 30500.
Saved 100 user profiles. Current Index: 30600.
Saved 100 user profiles. Current Index: 30700.
Saved 100 user profiles. Current Index: 30800.
Saved 100 user profiles. Current Index: 30900.
Không thể truy cập trang web, mã trạng thái: 500
Không thể truy cập trang web, mã trạng thái: 500
Không thể truy cập trang web, mã trạng thái: 500
Không thể truy cập trang web, mã trạng thái: 500
Không thể truy cập trang web, mã trạng thái: 500
Không thể truy cập trang web, mã trạng thái: 500
Không thể truy cập trang web, mã trạng thái: 500
Không thể truy cập trang web, mã trạng thái: 500
Không thể truy cập trang web, mã trạng thái: 500
Không thể truy cập trang web, mã trạng thái: 500
URL: https://bookcrossing.com/mybookshelf/V4Vicki/ se duoc bo qua.
Saved 99 user profiles. Current Index: 31000.
Saved 100 user profiles. Current Index: 31100.
Saved 100 user profiles. Current Index: 31200.
Saved 100 user profile

In [ ]:
# read user ratings
user_profiles_df = pd.read_csv('user_profiles.csv')
print(user_profiles_df.shape)
user_profiles_df

(23306, 6)


,user_url,image_url,name,area,age,join_date
0,/mybookshelf/_Figment_/,https://bookcrossing.com/images/memberpics/412...,_Figment_,"Regina, Saskatchewan Canada",47.0,"Saturday, April 2, 2005"
1,/mybookshelf/Ella-the-Elf/,https://bookcrossing.com/images/profile-image-...,Ella-the-Elf,"Vancouver, British Columbia Canada",37.0,"Thursday, January 9, 2003"
2,/mybookshelf/AnonymousFinder/,https://bookcrossing.com/images/memberpics/219...,AnonymousFinder,"Joined Tuesday, August 26, 2003",NaN,"Tuesday, August 26, 2003"
3,/mybookshelf/redhead344/,https://bookcrossing.com/images/profile-image-...,redhead344,Age 37,37.0,"Friday, November 15, 2002"
4,/mybookshelf/cestmoi/,https://bookcrossing.com/images/memberpics/118...,cestmoi,"Hamilton, Ontario Canada",56.0,"Sunday, October 13, 2002"
...,...,...,...,...,...,...
23301,/mybookshelf/Anneloeskrul/,https://bookcrossing.com/images/profile-image-...,Anneloeskrul,"Joined Sunday, August 24, 2008",NaN,"Sunday, August 24, 2008"
23302,/mybookshelf/siwebster/,https://bookcrossing.com/images/profile-image-...,siwebster,"St. Louis, Missouri USA",42.0,"Friday, May 12, 2006"
23303,/mybookshelf/gzola/,https://bookcrossing.com/images/profile-image-...,gzola,"Montgomery, Alabama USA",NaN,"Tuesday, July 23, 2002"
23304,/mybookshelf/angiepony/,https://bookcrossing.com/images/profile-image-...,angiepony,"Spokane, Washington USA",46.0,"Tuesday, July 23, 2002"


In [ ]:
error_user_urls_df = pd.read_csv('error_user_urls_profiles.csv')
print(error_user_urls_df.shape)
error_user_urls_df